### 참고

- 넷플릭스
    - 슬로건 : 모든 것이 추천이다
    - https://medium.com/netflixtechblog/
    - 넷플릭스의 추천 알고리즘 구현에 대한 기술 블로그 공개

# 1. 연구 목표

- 사용자 평점 데이터를 기반으로 사용자를 특정을 예측하여 추천시스템 구축
- 실제, OTT, 쇼핑몰에서 회원가입시 추천알고리즘으로 사용된다
- 회귀 처리, 회귀 평가, 추천시스템에 대한 이해
- FastFM(thrid party 알고리즘 사용 -> 인수분해머신 기능 지원)
    - 윈도우에서 컴파일 후 설치가 불가하므로, 리눅스에 설치하여 개발을 진행

# 2. 데이터 수집/확보

- ml-100k.zip 파일 제공
    - 미국에 있는 movelens라는 사이트에서 제공
- 영화 정보 데이터
    - 고객 정보
    - 영화 정보
    - 평점 정보

In [1]:
import pandas as pd

In [2]:
#u.user 라는 데이터에 컬럼이 없다보니, 첫번째 데이터가 컬럼이 되었다
#이를 예방하기 위해서는, 컬럼을 지정한다
cols=['uid','age','m','job','zip_code']
users = pd.read_csv('./table/ml-100k/u.user',sep='|', names=cols)
users.head(1)

,uid,age,m,job,zip_code
0,1,24,M,technician,85711


In [3]:
users.tail(1)

,uid,age,m,job,zip_code
942,943,22,M,student,77841


In [4]:
#고객 943의 데이터
users.shape

(943, 5)

In [5]:
#영화 정보 로드
m_cols=['mid','title','release_date','video_release_date','imdb_url']
#영어권이면 utf-8에서 에러나면 latin1으로 사용
#원본 데이터의 컬럼이 많은데 부분만 쓰고 싶다면, usecols를 적용
movies = pd.read_csv('./table/ml-100k/u.item',sep='|',encoding='latin1', names=m_cols,usecols=range(5))#, names=cols)
movies.head(2)

,mid,title,release_date,video_release_date,imdb_url
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...


In [6]:
movies.shape

(1682, 5)

In [7]:
movies.tail(2)

,mid,title,release_date,video_release_date,imdb_url
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...
1681,1682,Scream of Stone (Schrei aus Stein) (1991),08-Mar-1996,NaN,http://us.imdb.com/M/title-exact?Schrei%20aus%...


In [8]:
movies.head(2)

,mid,title,release_date,video_release_date,imdb_url
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...


In [9]:
movies.shape

(1682, 5)

In [10]:
#평점 정보 : u.data
r_cols=['uid','mid','rating','unix_timestamp']
ratings=pd.read_csv('./table/ml-100k/u.data', sep='\t', names=r_cols)
ratings.head(1)

,uid,mid,rating,unix_timestamp
0,196,242,3,881250949


In [11]:
ratings.tail(1)

,uid,mid,rating,unix_timestamp
99999,12,203,3,879959583


In [12]:
ratings.shape

(100000, 4)

In [13]:
ratings.unix_timestamp, ratings['unix_timestamp']

(0        881250949
 1        891717742
 2        878887116
 3        880606923
 4        886397596
            ...    
 99995    880175444
 99996    879795543
 99997    874795795
 99998    882399156
 99999    879959583
 Name: unix_timestamp, Length: 100000, dtype: int64, 0        881250949
 1        891717742
 2        878887116
 3        880606923
 4        886397596
            ...    
 99995    880175444
 99996    879795543
 99997    874795795
 99998    882399156
 99999    879959583
 Name: unix_timestamp, Length: 100000, dtype: int64)

In [14]:
#unix_timestamp는 1970년 1월 1일 00시 00분 00초 부터 현재까지 경과된 시간 + 9:00
#그래서 현재시간, 특정시간을 특정하기가 어렵다 -> 해석이 어렵다 -> 시간형식 변경
ratings['data']=pd.to_datetime(ratings.unix_timestamp, unit='s')

In [15]:
ratings.head(2)

,uid,mid,rating,unix_timestamp,data
0,196,242,3,881250949,1997-12-04 15:55:49
1,186,302,3,891717742,1998-04-04 19:22:22


In [16]:
# 3. 데이터 준비/품질향상/전처리
# 4. 데이터 분석/통계적, 시각적...

In [17]:
#평점과 영화데이터 합치기 (10000,9)
#merge를 수행할때 중복되는 컬럼을 생략하면 알아서 찾아서 수행된다
#단, 컬럼이 2개이상이면 검토가 필요
movies_ratings=pd.merge(movies,ratings,on='mid')
movies_ratings.shape

(100000, 9)

In [18]:
movies_ratings.head(2)

,mid,title,release_date,video_release_date,imdb_url,uid,rating,unix_timestamp,data
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,308,4,887736532,1998-02-17 17:28:52
1,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,287,5,875334088,1997-09-27 04:21:28


In [19]:
#movies_ratings, users->합치기
movie_lens=pd.merge(movies_ratings, users)
movie_lens.shape

(100000, 13)

In [20]:
movie_lens.head(2)

,mid,title,release_date,video_release_date,imdb_url,uid,rating,unix_timestamp,data,age,m,job,zip_code
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,308,4,887736532,1998-02-17 17:28:52,60,M,retired,95076
1,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,308,5,887737890,1998-02-17 17:51:30,60,M,retired,95076


- 데이터를 모두 병합하였다. 평점이나 회원을 중심으로 중복적인 데이터가 많다
- 데이터가 적으면 크게 문제 없으나, 크면 메모리를 많이 사용할 수도 있다(검토)

In [21]:
# 평가를 가장 많이 받은 영화 상위 10개 출력하시오
movie_lens['title'].value_counts()[:10]

Star Wars (1977)                 583
Contact (1997)                   509
Fargo (1996)                     508
Return of the Jedi (1983)        507
Liar Liar (1997)                 485
English Patient, The (1996)      481
Scream (1996)                    478
Toy Story (1995)                 452
Air Force One (1997)             431
Independence Day (ID4) (1996)    429
Name: title, dtype: int64

In [22]:
movie_lens['title'].value_counts()[-10:]
#평점을 받은 횟수가 적은경우, 우연히 평점이 좋을 수도 있다
#데이터를 조정할때, 특정 회수 이상 평가한 작품만 고려하겠다

MURDER and murder (1996)                        1
Lashou shentan (1992)                           1
Leopard Son, The (1996)                         1
Stefano Quantestorie (1993)                     1
King of New York (1990)                         1
Spanish Prisoner, The (1997)                    1
Power 98 (1995)                                 1
New Age, The (1994)                             1
Eye of Vichy, The (Oeil de Vichy, L') (1993)    1
He Walked by Night (1948)                       1
Name: title, dtype: int64

In [23]:
import numpy as np

- 제목기준으로 데이터를 분류
- 평가의 개수, 평균 평점이 들어가있는 DF
- 인덱스:title, 컬럼:개수, 평균평점<-만드는 과정에서 컬럼의 레벨이 1이상이 되고 관계없음

In [24]:
#groupby(컬럼) : 해당 컬럼이 인덱스로 이동
#agg({컬럼:[값처리함수...]}):컬럼에 처리함수개수대로 설정되서 값이 자동 처리
movie_state=movie_lens.groupby('title').agg({'rating':[np.size,np.mean]})
movie_state

rating          
                                        size      mean
title                                                 
'Til There Was You (1997)                  9  2.333333
1-900 (1994)                               5  2.600000
101 Dalmatians (1996)                    109  2.908257
12 Angry Men (1957)                      125  4.344000
187 (1997)                                41  3.024390
...                                      ...       ...
Young Guns II (1990)                      44  2.772727
Young Poisoner's Handbook, The (1995)     41  3.341463
Zeus and Roxanne (1997)                    6  2.166667
unknown                                    9  3.444444
Á köldum klaka (Cold Fever) (1994)         1  3.000000

[1664 rows x 2 columns]

In [25]:
#이 케이스에서는 피벗의 표현이 좀더 복잡할수 있어서 배제
#movie_lens.pivot_table()

- 평점을 받은 개수가 1개인 경우, 소수의 평가를 받은 영화
- 평균의 수가 적으면 잡음의 개입이 여지가 많다
- 100건 기준으로 100개 이상만 대상으로 처리(임계값)

In [26]:
#차후 변경함으로써 성능을 다르게 낼 수 있다
limit_std_value = 100
condition=movie_state['rating']['size']>=limit_std_value
condition[:2]

title
'Til There Was You (1997)    False
1-900 (1994)                 False
Name: size, dtype: bool

In [27]:
movie_state[condition]

rating          
                                               size      mean
title                                                        
101 Dalmatians (1996)                           109  2.908257
12 Angry Men (1957)                             125  4.344000
2001: A Space Odyssey (1968)                    259  3.969112
Absolute Power (1997)                           127  3.370079
Abyss, The (1989)                               151  3.589404
...                                             ...       ...
Willy Wonka and the Chocolate Factory (1971)    326  3.631902
Wizard of Oz, The (1939)                        246  4.077236
Wrong Trousers, The (1993)                      118  4.466102
Young Frankenstein (1974)                       200  3.945000
Young Guns (1988)                               101  3.207921

[338 rows x 2 columns]

In [28]:
#정렬->평균이 내림차순으로 정렬하여 상위 5개만 출력
tmp=movie_state[condition].sort_values(by=[('rating','mean')], ascending=False)
tmp[:5]

rating          
                                   size      mean
title                                            
Close Shave, A (1995)               112  4.491071
Schindler's List (1993)             298  4.466443
Wrong Trousers, The (1993)          118  4.466102
Casablanca (1942)                   243  4.456790
Shawshank Redemption, The (1994)    283  4.445230

In [29]:
#영화 1682개 중에 임계값을 통과한 대상은 338개이다
#이를 통해 평점을 적게 받은 영화의 통계적인 추정도 가능
tmp.shape, movies.shape

((338, 2), (1682, 5))

In [30]:
#간단한 시각화
#영화별 평점 개수 : x축
#평가 회수 : y축
from matplotlib import pyplot as plt

%matplotlib inline

In [31]:
# 히스토그램
plt.style.use('ggplot')
movie_lens.groupby('uid').size().sort_values(ascending=False)

uid
405    737
655    685
13     636
450    540
276    518
      ... 
36      20
34      20
685     20
441     20
202     20
Length: 943, dtype: int64

In [32]:

movie_lens.groupby('uid').size().sort_values(ascending=False).hist()
#사용자의 평가 횟수에 대한 성향
#빈도가 점점 낮아진다->길게 꼬리를 늘어뜨리는 모양->롱테일분포
#'지프의 법칙'을 따른 굴곡 모양이다
#영화의 평가가 많으면, 그 사용자들 중에는 1회성 평가 회수도 많다
#스타워즈1977년 작품평가가 583회 수행되었는데, 이 중에는 이 영화만 평가한 유저도 다수 존재
# (현 데이터에서는 최저 평가 회수가 20)

SyntaxError: invalid syntax (<ipython-input-32-651e830fbcb2>, line 7)

In [33]:
#사용자별 평가 회수, 평균
user_state=movie_lens.groupby('uid').agg({'rating':[np.size,np.mean]})
user_state.head()

rating          
      size      mean
uid                 
1      272  3.610294
2       62  3.709677
3       54  2.796296
4       24  4.333333
5      175  2.874286

In [34]:
user_state.shape

(943, 2)

In [35]:
user_state.describe()
#최저값:1.49, 최대:4.87, 25~75% 지점:3점대로 몰려있다
#평균 : 3.58 -> 어떤 영화든지 일반적으로 3점이상은 받는다
#3점 이하는 영화 자체에 문제가 있어 보인다
#최고, 최저는 편중되어 있다

rating            
             size        mean
count  943.000000  943.000000
mean   106.044539    3.588191
std    100.931743    0.445233
min     20.000000    1.491954
25%     33.000000    3.323054
50%     65.000000    3.620690
75%    148.000000    3.869565
max    737.000000    4.869565

### 5. 예측모델구축(머신러닝기반)

- 알고리즘 -> 인수분해 머신기능을 제공하는 FastFM이라는 모듈 사용
- FastFM
    - C++로 만들어진 libFM이라는 알고리즘을 래핑
    - libFM을 python으로 구성한것이 FastFM
    - 기능
        - 행렬 인수 분해라는 기능을 일반화하여서 제공 -> 차원축소기법
        - 범주형 변수를 파생변수로 변환하여 범주간의 상호작용성을 계산
        - 특징 간 영향을 주고 받은 상호작용 개념을 계산

- fastFM 제공 알고리즘
    - ALS : 교대 최소 제곱법
        - 장점: 예측시간빠름, SGD에 비해 하이퍼파라미터 작다
        - 단점: 규제처리가 필요
    - SGD : 확률적 경사 하강법
        - 장점: 예측시간빠름, 빅데이터를 빠르게 처리학습 할수 있다
        - 단점: 하이퍼파라미터가 많다, 규제처리 필요
    - MCMC:마르코프 연쇄 몬테카를로
        - 장점: 하이퍼파라미터가 작다, 자동규제
        - 단점: 학습시간 느리다

In [ ]:
#아마존 EC2 서버에 접속하였다
#febric3을 이용하여 이하 과정을 자동화 할수있다

#리눅스상에서 루트 권한 획득
ubuntu:$ sudo su
    
#리눅스의 현재 설치된 패키지 등을 최신으로 업그레이드
root:$ apt-get update && apt-get -y upgrade

#파이썬 확인
$ python -V
$ python3 -V
    
#패키지 설치(파이썬이 설치되어 있으면 파이썬 부분은 제외)
$ apt-get -y install python3-dev python3-pip git nano wget unzip libopenblas-dev
#파이썬이 있으면 이하만 설치
$ apt-get -y install git nano wget unzip libopenblas-dev

#fastFM 소스 다운로드
$ git clone --recursive https://github.com/ibayer/fastFM.git
$ cd fastFM

# 컴파일을 수행하기 전에 python 모듈 설치
# 내용확인
$ cat requirements.txt
$ pip3 install -r ./requirements.txt

#컴파일 -> 마지막 부분에 error가 보여도 무시
$ PYTHON=python3 make

#fastFM 설치
$ pip3 install .

#확인
$python3
>>> from fastFM import als
>>> exit()

# 개발에 필요한 패키지 설치
$ pip3 install pandas matplotlib jupyter


#옵션 삭제
$ rm -r -f dev

#root 계정 오프
$exit

#작업폴더 생성
ubuntu $

# 주피터 가동
$ jupyter notebook --ip=0.0.0.0 --port=8888 --allow-root --no-browser
$ jupyter notebook --ip=0.0.0.0 --allow-root --no-browser

http://ip-15-164-232-129:8888/?token=3186c7951cfef769f8999912a53e605b78b17c1bf80e88da
http://15-164-232-129:8888/?token=7176d151ed5f5b04b5c5a156f136d1ce2515bf689d6b230f

- 여기서부터는 리눅스에서 운용되므로 현 파일은 종료
    - 단, 리눅스 서버에서 주피터를 새로 가동했으면 토큰은 신규로 입력해야 한다
<a href='http://15.164.232.129:8888/'>이동</a>

In [36]:
from sklearn.feature_extraction import DictVectorizer
import numpy as np

In [37]:
# DictVectorizer : 문자열만 벡터화 처리
v = DictVectorizer()

In [45]:
#더미 데이터
#사용자 ID, 사용자가 평가한 영화 ID, 사용자의 나이
train=[
    {'uid':'1','mid':'5','age':19},
    {'uid':'2','mid':'43','age':33},
    {'uid':'3','mid':'20','age':55},
    {'uid':'4','mid':'10','age':20},
]

In [46]:
X = v.fit_transform(train)
#수치는 그대로 배치, 문자열이 들어간 데이터는 벡터화로 처리
#문자열은 범주형으로 보고 해당 케이스별로 배치하여 0혹은 1로 표시
X.toarray()

array([[19.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.],
       [33.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
       [55.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.],
       [20.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [40]:
#ALS를 이용해서 처리
from fastFM import als
from sklearn.model_selection import learning_curve

In [41]:
#더미데이터 중 20살 유저 1번은 영화 5번에 평점 5.0을 부여했다
#유저별로 부여한 평점
y=np.array([5.0,1.0,2.0,4.0])


In [47]:
#ALS를 이용하여 fastFM의 회귀모델을 초기화후 학습을 진행
#하이퍼파라미터는 임시값 부여
fm=als.FMRegression(n_iter=1000, init_stdev=0.1, rank=2, l2_reg_w=0.1, l2_reg_V=0.5)

In [48]:
fm.fit(X,y)

FMRegression(init_stdev=0.1, l2_reg=0, l2_reg_V=0.5, l2_reg_w=0.1, n_iter=1000,
             random_state=123, rank=2)

In [49]:
# 예측
# 새로운 유저가 신규 가입을 했다. uid가 5번이 되었다.
# 나이가 24인 유저가 영화 10번에 대해 내릴 평점을 예측해라
# {'uid':'5','mid':'10','age':24}
fm.predict(v.transform({'uid':'5','mid':'10','age':24}))

array([3.60775939])

- 제공되는 데이터는 ua.base, ua.test는 훈련용과 테스트용이 구분되어 있다
- 이를 읽어서 자료구조를 만드는 형태로 처리하는 함수 구현

In [55]:
def loadData(fileName, path='./table/ml-100k/'):
    data = list() #학습용 데이터 형태
    y = list() #답안(평점)
    #데이터 추출 및 구조 작업
    with open(path + fileName) as f: #파일 오픈
        #한줄식 읽어서 처리
        for line in f:
            #데이터 한줄을 구분해서 추출
            uid, mid, rating, ts = line.split('\t')
            #유저아이디와 영화아이디 추가
            data.append({'uid':str(uid),'mid':str(mid)})
            #점수 추가
            y.append(float(rating))
            
    return data, np.array(y)

#훈련용 데이터 획득
dev_data, y_dev = loadData('ua.base')

In [57]:
len(dev_data), y_dev.shape, y_dev[:4]

(90570, (90570,), array([5., 3., 4., 3.]))

In [59]:
#테스트용 데이터 획득
test_data, y_test=loadData('ua.test')
len(test_data), y_test.shape, y_test[:4]

(9430, (9430,), array([4., 4., 4., 3.]))

In [62]:
# 해당 함수 확장 loadDataEx
# 평가에 참가한 유저들의 ID만 모은 데이터셋, 영화의 ID만 모은 데이터셋
def loadDataEx(fileName, path='./table/ml-100k/'):
    data = list() 
    y = list()
    users = set() #중복데이터를 제거하는 자료구조 생성
    movies = set()
    with open(path + fileName) as f:
        for line in f:
            uid, mid, rating, ts = line.split('\t')
            data.append({'uid':str(uid),'mid':str(mid)})
            y.append(float(rating))
            #평가 데이터에서 uid, mid를 추가
            users.add(uid)
            movies.add(mid)
    return data, np.array(y), users, movies

In [63]:
#훈련용 데이터 획득
dev_data, y_dev, dev_users, dev_movies = loadDataEx('ua.base')
#테스트용 데이터 획득
test_data, y_test, test_users, test_movies =loadDataEx('ua.test')


In [64]:
#훈련용 및 테스트용에 참가한 유저수는 동수
#영화수는 훈련용이 테스트용보다 많다
len(dev_users), len(dev_movies), len(test_users), len(test_movies)

(943, 1680, 943, 1129)

- 데이터 벡터화 처리
- 사전 데이터들을 수치라도 다 문자로 처리
- 평점만 빼고, uid, mid값은 현재 문자열이다

In [69]:
#확인
dev_data[:1]
y_dev[:1]

array([5.])

In [66]:
#훈련용 데이터의 벡터화
X_dev=v.fit_transform(dev_data)
X_dev.shape

(90570, 2623)

In [67]:
type(X_dev)

scipy.sparse.csr.csr_matrix

In [68]:
#테스트데이터의 벡터화
X_test=v.fit_transform(test_data)
X_test.shape

(9430, 2072)

In [71]:
#표준편차 확인 -> 회귀에서 평가지수 -> 평균제곱근오차를 게산할 때 평가 잣대
np.std(y_dev), np.std(y_test)

(1.1260664426539722, 1.120180145761465)

In [72]:
from sklearn.model_selection import train_test_split

#9:1 비율(임의설정)
#randomstate -> 항상동일하게 섞이게
X_train, X_dev_test, y_train, y_dev_test=train_test_split(X_dev, y_dev, test_size=0.1, random_state=42)

In [73]:
X_train.shape, X_dev_test.shape

((81513, 2623), (9057, 2623))

- 데이터는 모두 준비되었다.
- 회귀 모델에 학습 및 예측을 수행하기 위해 데이터의 형태를 알고리즘에 맞춰서 구성하였다
- 알고리즘 선택
    - mcmc
        - 학습 및 예측 수행
        - 시각화를 통해 수렴해하는 과정 확인
        - 평균제곱근오차 및 손실함수를 이용하여 평가
            - 테스트데이터를 이용한 성능 측정
        - 하이퍼파라미터 활용(고정)하여 평점의 정규화 처리, 성능 평가
        - 기타 정보 추가